In [1]:
# Cargo los datos
import pandas as pd

df_train = pd.read_csv('data/train_timeseries_interpolated.csv')
df_val = pd.read_csv('data/val_timeseries_interpolated.csv')
df_test = pd.read_csv('data/test_timeseries_interpolated.csv')


In [2]:
df_soil = pd.read_csv('data/soil_data.csv')
# Merge de los datos de suelo
df_train = df_train.merge(df_soil, on='fips', how='left')
df_val = df_val.merge(df_soil, on='fips', how='left')
df_test = df_test.merge(df_soil, on='fips', how='left')

In [5]:
df_train.columns

Index(['fips', 'date', 'PRECTOT', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET',
       'T2M_MAX', 'T2M_MIN', 'T2M_RANGE', 'TS', 'WS10M', 'WS10M_MAX',
       'WS10M_MIN', 'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN',
       'WS50M_RANGE', 'score', 'score_interpolated',
       'score_final_interpolated', 'lat', 'lon', 'elevation', 'slope1',
       'slope2', 'slope3', 'slope4', 'slope5', 'slope6', 'slope7', 'slope8',
       'aspectN', 'aspectE', 'aspectS', 'aspectW', 'aspectUnknown', 'WAT_LAND',
       'NVG_LAND', 'URB_LAND', 'GRS_LAND', 'FOR_LAND', 'CULTRF_LAND',
       'CULTIR_LAND', 'CULT_LAND', 'SQ1', 'SQ2', 'SQ3', 'SQ4', 'SQ5', 'SQ6',
       'SQ7'],
      dtype='object')

In [8]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Preparar los datos para la clusterización
X = df_train[['PRECTOT', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET',
       'T2M_MAX', 'T2M_MIN', 'T2M_RANGE', 'TS', 'WS10M', 'WS10M_MAX',
       'WS10M_MIN', 'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN',
       'WS50M_RANGE', 'score', 'score_interpolated',
       'score_final_interpolated', 'lat', 'lon', 'elevation', 'slope1',
       'slope2', 'slope3', 'slope4', 'slope5', 'slope6', 'slope7', 'slope8',
       'aspectN', 'aspectE', 'aspectS', 'aspectW', 'aspectUnknown', 'WAT_LAND',
       'NVG_LAND', 'URB_LAND', 'GRS_LAND', 'FOR_LAND', 'CULTRF_LAND',
       'CULTIR_LAND', 'CULT_LAND', 'SQ1', 'SQ2', 'SQ3', 'SQ4', 'SQ5', 'SQ6',
       'SQ7']].copy()

# Normalizar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplicar K-means
# Puedes ajustar el número de clusters según tus necesidades
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# Manejo los valores nan
X_scaled = np.nan_to_num(X_scaled)
df_train['cluster'] = kmeans.fit_predict(X_scaled)

# Analizar los resultados
cluster_centroids = scaler.inverse_transform(kmeans.cluster_centers_)
cluster_sizes = df_train['cluster'].value_counts().sort_index()

print("Tamaño de los clusters:")
print(cluster_sizes)

print("\nCentroides de los clusters:")
for i, centroid in enumerate(cluster_centroids):
    print(f"Cluster {i}: {centroid}")

# Seleccionar un FIPS representativo por cluster
representative_fips = df_train.groupby('cluster').apply(
    lambda x: x.loc[x['distancia_al_centroide'].idxmin(), 'fips']
).reset_index(name='fips_representativo')

print("\nFIPS representativos por cluster:")
print(representative_fips)

# # Guardar los resultados
# df.to_csv('dataset_clusterizado.csv', index=False)
# representative_fips.to_csv('fips_representativos.csv', index=False)

Tamaño de los clusters:
cluster
0    1758116
1    1411216
2    1441552
3     935888
4     131544
Name: count, dtype: int64

Centroides de los clusters:
Cluster 0: [ 3.22405895e+00  9.89716984e+01  1.01971707e+01  1.75728868e+01
  1.25563623e+01  1.25641791e+01  2.35170442e+01  1.21323868e+01
  1.13846638e+01  1.75803673e+01  2.33620614e+00  3.36941570e+00
  1.31402227e+00  2.05539673e+00  4.22253045e+00  6.07422371e+00
  2.31306145e+00  3.76115576e+00  8.37033065e-01  6.26548490e-01
  6.22222020e-01  3.65378760e+01 -8.59285545e+01  2.22935636e+02
  1.94394765e-02  1.95956870e-01  4.10016152e-01  2.51787579e-01
  6.81292895e-02  4.64429365e-02  5.24130535e-03  5.53338524e-04
  1.76943143e-01  2.08166006e-01  1.91958734e-01  2.05102718e-01
  2.15396346e-01  1.45194406e+00  4.16869557e-01  3.83856680e+00
  1.14971009e+01  6.11618588e+01  2.03052854e+01  1.01507707e+00
  2.13203625e+01  2.37477044e+00  1.79449702e+00  1.10717322e+00
  1.15528706e+00  1.00776057e+00  1.00629661e+00  1.10081

KeyError: 'distancia_al_centroide'

In [12]:
df_train_clusters=df_train[['fips', 'cluster']].value_counts().sort_index()

In [15]:
df_train_clusters

fips   cluster
1001   0          1681
       1            34
       2           112
1003   0           126
       1          1620
                  ... 
56039  3          1827
56041  2           354
       3          1473
56043  2           133
       3          1694
Name: count, Length: 7685, dtype: int64

In [18]:
df_train_clusters.groupby('fips').first()

fips
1001     1681
1003      126
1005     1785
1007     1825
1009     1727
         ... 
56035     736
56037     218
56039    1827
56041     354
56043     133
Name: count, Length: 3108, dtype: int64

In [21]:
df_train_clusters.groupby(['cluster','fips']).agg(pd.Series.mode)

cluster  fips 
0        1001     1681
         1003      126
         1005     1785
         1007     1825
         1009     1727
                  ... 
4        55071    1827
         55079    1827
         55089    1827
         55101    1827
         55117    1827
Name: count, Length: 7685, dtype: int64